In [3]:
from scipy.spatial.distance import pdist, squareform
import umap.umap_ as umap
import pandas as pd
import hdbscan
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import os

In [22]:
def build_umap_plot_2d_clusters_from_metadata(metadata_cluster_file):
    """
    Builds a 2D UMAP plot from metadata with 'UMAP 1', 'UMAP 2', and 'cluster' columns.
    """
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px

    # Load metadata
    metadata = pd.read_csv(metadata_cluster_file, sep='\t')

    import colorcet as cc  # for colorblind-friendly palettes
    import random

    # Combine multiple colorblind-safe palettes
    safe_colors = (
        cc.glasbey[:25]  # Glasbey is great for many distinguishable categories
    )

    # Shuffle for aesthetics if desired
    random.seed(42)  # ensure reproducibility
    random.shuffle(safe_colors)

    unique_clusters = sorted(metadata['cluster'].unique())
    cluster_color_dict = {
        cluster: ('gainsboro' if cluster == -1 else safe_colors[i % len(safe_colors)])
        for i, cluster in enumerate(unique_clusters)
    }
    metadata['cluster_color'] = metadata['cluster'].map(cluster_color_dict)


    # Build plot
    fig = go.Figure()

    for cluster_id in unique_clusters:
        cluster_df = metadata[metadata['cluster'] == cluster_id]
        # cluster_name = cluster_df['clusterNameReactome'].iloc[0] if cluster_id != -1 else 'Noise'
        color = cluster_color_dict[cluster_id]
        # legend_name = f"{cluster_name}" if cluster_id != -1 else 'Noise'

        # # Marker outline only for drug targets (not visible in legend)
        # marker_lines = [
        #     dict(width=1, color='black') if val != 0 else dict(width=0)
        #     for val in cluster_df['geneticAD']
        # ]

        fig.add_trace(go.Scatter(
            x=cluster_df['UMAP 1'],
            y=cluster_df['UMAP 2'],
            mode='markers',
            # name=legend_name,
            marker=dict(
                size=5,
                color=color,
                line=dict(width=0),  # Disable uniform outline to prevent black circles in legend
                opacity=0.7
            ),
            customdata=cluster_df[['cluster', 'approvedSymbol']],
            hovertemplate=(
                "Cluster %{customdata[0]}: %{customdata[1]}<br>" +
                "approvedSymbol: %{customdata[2]}<extra></extra>"
            ),
            showlegend=True
        ))

        # Add labels for maxPhase > 2
        high_val_df = cluster_df[cluster_df['maxPhaseChEMBL'] > 2].copy()
        y_offset = 0.15
        fig.add_trace(go.Scatter(
            x=high_val_df['UMAP 1'],
            y=high_val_df['UMAP 2'] + y_offset,
            mode='text',
            text=high_val_df['approvedSymbol'],
            textposition='top center',
            textfont=dict(color='black', size=12),
            showlegend=False,
            hoverinfo='skip'
        ))

    # # Add dummy legend entry for drug targets
    # fig.add_trace(go.Scatter(
    #     x=[None],
    #     y=[None],
    #     mode='markers',
    #     marker=dict(
    #         symbol='circle-open',
    #         color='black',
    #         size=5,
    #         line=dict(width=1)
    #     ),
    #     # name='– drug target (Pharmaprojects)',
    #     name='– NeuroFlux hit',
    #     showlegend=True
    # ))

    fig.update_layout(
        xaxis=dict(title='UMAP1', showgrid=False, zeroline=False, constrain='domain', ticks='outside', showline=True, mirror=True),
        yaxis=dict(title='UMAP2', showgrid=False, zeroline=False, scaleanchor='x', scaleratio=1, constrain='domain', ticks='outside', showline=True, mirror=True),
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40),
        legend=dict(title='Cluster', itemsizing='constant'),
        height=700,
        width=800,
    )

    return fig

In [23]:
metadata_cluster_file = '/Users/polina/Pathwaganda/data/umap/test/metadata_clusters_poincare_fast_ge.tsv'
fig = build_umap_plot_2d_clusters_from_metadata(metadata_cluster_file)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[-1, 'BAK1'],
                                   [-1, 'NFATC2'],
                                   [-1, 'FLNA'],
                                   ...,
                                   [-1, 'ITK'],
                                   [-1, 'ORMDL3'],
                                   [-1, 'PTPN13']], dtype=object),
              'hovertemplate': ('Cluster %{customdata[0]}: %{cu' ... '{customdata[2]}<extra></extra>'),
              'marker': {'color': 'gainsboro', 'line': {'width': 0}, 'opacity': 0.7, 'size': 5},
              'mode': 'markers',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': ('inIOQ1ASFkC1GDxM+xYfQFwWuOEcch' ... 'Rb5xvAMxtkkpH7IEDc4WU8AFgEQA=='),
                    'dtype': 'f8'},
              'y': {'bdata': ('vCTOiqjJA0DWxtgJLyEaQAma4OUzag' ... 'JT+tS/0ZLH0/ITIECGIXL6ek4XQA=='),
                    'dtype': 'f8'}},
             {'hoverinfo': 'skip',
              'mode': 'text',
              'showlegend': False,
              'text': array([], dtype=object),
              'textfont': {'color': 'black', 'size': 12},
              'textposition': 'top center',
              'type': 'scatter',
              'x': array([], dtype=float64),
              'y': array([], dtype=float64)},
             {'customdata': array([[0, 'IL21R'],
                                   [0, 'IL7'],
                                   [0, 'IL12A'],
                                   [0, 'CRLF2'],
                                   [0, 'STAT4']], dtype=object),
              'hovertemplate': ('Cluster %{customdata[0]}: %{cu' ... '{customdata[2]}<extra></extra>'),
              'marker': {'color': '#a17569', 'line': {'width': 0}, 'opacity': 0.7, 'size': 5},
              'mode': 'markers',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'rcH7qlwAKECDUrRyL4gnQLoT7L/O4SdAE38Udeb+J0Cyu0BJgS0oQA==', 'dtype': 'f8'},
              'y': {'bdata': 'uW3fo/5eOkBK0F/oEZs6QK7YX3ZPbjpAie3uAbpfOkDICn4bYkg6QA==', 'dtype': 'f8'}},
             {'hoverinfo': 'skip',
              'mode': 'text',
              'showlegend': False,
              'text': array([], dtype=object),
              'textfont': {'color': 'black', 'size': 12},
              'textposition': 'top center',
              'type': 'scatter',
              'x': array([], dtype=float64),
              'y': array([], dtype=float64)},
             {'customdata': array([[1, 'ELF3'],
                                   [1, 'RAP1GDS1'],
                                   [1, 'MSI2'],
                                   [1, 'RRAS2'],
                                   [1, 'BMPR1A'],
                                   [1, 'NF2'],
                                   [1, 'ARHGEF10'],
                                   [1, 'RHOH'],
                                   [1, 'ARHGEF10L'],
                                   [1, 'LATS2'],
                                   [1, 'SPEN'],
                                   [1, 'SUFU'],
                                   [1, 'SPOP'],
                                   [1, 'IKZF1'],
                                   [1, 'GLI1']], dtype=object),
              'hovertemplate': ('Cluster %{customdata[0]}: %{cu' ... '{customdata[2]}<extra></extra>'),
              'marker': {'color': '#005659', 'line': {'width': 0}, 'opacity': 0.7, 'size': 5},
              'mode': 'markers',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': ('+5XOh2fhIEAKaY1BJxwfQKPLm8O12h' ... 'fFgaF4HkAgJAuYwBUhQAHs7eBxCx9A'),
                    'dtype': 'f8'},
              'y': {'bdata': ('wW7YtiijQ0ANMzSeCPhDQGKdKt8zAE' ... '9U+3QMREDy0k1iEJZDQDJ3LSEf+kNA'),
                    'dtype': 'f8'}},
             {'hoverinfo': 'skip',
              'mode': 'text',
              'showlegend': False,
              'text': array([], dtype=object),
              'textfont': {'color': 'black', 'size': 12},
         